<a href="https://colab.research.google.com/github/lucasmesq1/Projeto_Dados_Med/blob/main/BigData_Project_Med.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Apresentação**

##**1. Bibliotecas Importadas**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import seaborn as sns
plt.style.use('ggplot')


##**2. Lendo o Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Definindo o caminho dos datasets a utilizar
anvisa2019 = '/content/drive/MyDrive/projeto_bigdata/anvisa2019.csv'
anvisa2021 = 'https://drive.google.com/drive/folders/1SV6hbRJ4g050YiUEgGnyqTyecqrU20wE?usp=drive_link'
print("Datasets acessados via google drive")

In [ ]:
# Defina o tamanho do chunk
chunksize = 10000

# lista para armazenar os dados processados
df_old = []

# Leia o arquivo em pedaços
for chunk in pd.read_csv("anvisa2021.csv", chunksize=chunksize):

    df_old.append(chunk)  # Armazena o chunk se precisar de todos os dados

# Concatenar um DataFrame único no final
df_new = pd.concat(df_old, ignore_index=True)

print(df_new.head())

In [ ]:
#Retorna o tamanho do DF, linhas x colunas
df_new.shape

In [ ]:
#Mostra um resumo conciso sobre o DF
df_new.info()

In [ ]:
#Gera descrições para as colunas (dados numéricos e categóricos)
df_new.describe(include='all')

In [ ]:
#Fornece uma contagem do número de valores nulos(ou ausentes) em cada coluna do DF
df_new.isnull().sum()

In [ ]:
#Calcular a porcentagem de valores nulos em cada coluna
df_new.isnull().sum()/df_new.shape[0]*100

In [ ]:
#Retorna o número total de linhas que são duplicadas em relação a linhas anteriores
df_new.duplicated().sum()

##**3. Tratando os Dados**

In [ ]:
#Removendo colunas
df = df_new.drop(columns=['ANO_VENDA','MES_VENDA','DCB','CONSELHO_PRESCRITOR','UF_CONSELHO_PRESCRITOR','CID10'])
df.head(3)

In [ ]:
#Percorre todas as colunas que contêm dados do tipo "object" (geralmente strings ou textos) e imprime a contagem de valores únicos para cada uma dessas colunas.
for i in df.select_dtypes(include='object').columns:
    print(df[i].value_counts())
    print('***' * 10)


In [ ]:
#Utilizado para obter estatísticas descritivas das colunas que contêm dados do tipo "object"
df.describe(include='object')

In [ ]:
#Histograma para a distribuição
import warnings
warnings.filterwarnings('ignore')

for i in df.select_dtypes(include='number').columns:
    sns.histplot(data=df, x=i)
    plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for i in df.select_dtypes(include='number').columns:
    sns.boxplot(data=df, x=i)
    plt.show()

**3.1 Missing Value**

In [ ]:
df.isnull().sum()

In [ ]:
colunas = ['SEXO','IDADE','UNIDADE_IDADE']
for col in colunas:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
from sklearn.impute import SimpleImputer
impute = SimpleImputer(strategy='mean')

In [ ]:
for i in df.select_dtypes(include='number').columns:
    df[i] = impute.fit_transform(df[[i]])

**3.2. Tratar Outliers**

In [ ]:
df.select_dtypes(include=['number'])

In [ ]:
#def tratar_outliers(df, colunas):
 #   for coluna in colunas:
        # Calcular Q1 e Q3
  #      Q1 = df[coluna].quantile(0.25)
   #     Q3 = df[coluna].quantile(0.75)
    #    IQR = Q3 - Q1

        # Definir limites
     #   limite_inferior = Q1 - 1.5 * IQR
      #  limite_superior = Q3 + 1.5 * IQR

        # Substituir outliers por NaN
       # df[coluna] = df[coluna].where((df[coluna] >= limite_inferior) & (df[coluna] <= limite_superior), other=pd.NA)

   # return df

# Tratando outliers nas colunas especificadas
#colunas_para_tratar = []
#df_tratado = tratar_outliers(df, colunas_para_tratar)

#print(df_tratado)

##**4. Análise Descritiva**

**4.1. Análise Univariada**

In [ ]:
f1 = df['UF_VENDA'].value_counts()

plt.figure(figsize=(10, 6))
f1.plot(kind='bar')
plt.title('Frequência para cada estado')
plt.xlabel('Estados(UF)')
plt.ylabel('Quantidade')
plt.xticks(rotation=45)
plt.show()

In [ ]:
f2 = df['TIPO_RECEITUARIO'].value_counts()

plt.figure(figsize=(10, 6))
f2.plot(kind='bar')
plt.title('Gráfico da frequência')
plt.xlabel('Municípios')
plt.ylabel('Quantidade')
plt.xticks(rotation=90)
plt.show()

In [ ]:
f3 = df['SEXO'].value_counts()

# Definindo cores específicas para masculino e feminino
cores = ['blue' if sexo == 1.0 else 'pink' for sexo in f3.index]

# Criando o gráfico de barras
plt.figure(figsize=(10, 6))
f3.plot(kind='bar', color=cores, edgecolor='black')
plt.title('Frequência por Sexo')
plt.xlabel('Feminino                                                           Masculino')
plt.ylabel('Quantidade')
plt.xticks(rotation=0)  # Rotação dos rótulos no eixo x
plt.grid(axis='y')  # Adiciona uma grade no eixo y
plt.show()

In [ ]:
# Filtrar para idades até 120
df = df[df['IDADE'] <= 120]

# Definir faixas etárias
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99']

# Criar uma coluna de faixas etárias
df['faixa_etaria'] = pd.cut(df['IDADE'], bins=bins, labels=labels, right=False)

# Contar a frequência de cada faixa etária
faixa_counts = df['faixa_etaria'].value_counts().sort_index()

# Criar o gráfico de barras
plt.bar(faixa_counts.index, faixa_counts.values)
plt.xlabel('Faixa Etária')
plt.ylabel('Frequência')
plt.title('Distribuição de Idades por Faixa Etária (até 120 anos)')
plt.xticks(rotation=45)  # Rotacionar os rótulos do eixo x para melhor legibilidade
plt.show()


**4.2. Análise Bivariada**

In [ ]:
df['LOCAL'] = df['MUNICIPIO_VENDA'] + ' - ' + df['UF_VENDA']

In [ ]:
df['AMOSTRA_VENDA'] = df['PRINCIPIO_ATIVO'].astype(str) + ' - ' + df['QTD_UNIDADE_FARMACOTECNICA'].astype(str)

In [ ]:
df['AMOSTRA_COM_LOCAL'] = df['AMOSTRA_VENDA'].astype(str) + ' - ' + df['LOCAL'].astype(str)

In [ ]:
# Função para filtrar dados relevantes de uma coluna
print("Filtro")
def filtrar_top_10_frequencias(df, coluna):
    frequencias = df[coluna].value_counts()
    top_10_valores = frequencias.nlargest(10).index
    df_filtrado = df[df[coluna].isin(top_10_valores)]
    return df_filtrado

Filtro


In [ ]:
# Tabela "LOCAL"
df_filtrado = filtrar_top_10_frequencias(df, 'LOCAL')

# Contar as frequências dos produtos filtrados
frequencias_filtradas = df_filtrado['LOCAL'].value_counts().reset_index()
frequencias_filtradas.columns = ['Produto', 'Frequência']

# Plotar o gráfico de barras usando seaborn
plt.figure(figsize=(10, 6))
sns.barplot(x='Produto', y='Frequência', data=frequencias_filtradas, palette='Blues_d')
plt.title('Maior Compra')
plt.xlabel('UF/Cidades')
plt.ylabel('Quantidade')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Tabela de "AMOSTRAS"
df_filtrado = filtrar_top_10_frequencias(df, 'AMOSTRA_VENDA')

# Contar as frequências dos produtos filtrados
frequencias_filtradas = df_filtrado['AMOSTRA_VENDA'].value_counts().reset_index()
frequencias_filtradas.columns = ['Produto', 'Quantidade']

# Plotar o gráfico de barras usando seaborn
plt.figure(figsize=(10, 6))
sns.barplot(x='Produto', y='Quantidade', data=frequencias_filtradas, palette='Blues_d')
plt.title('Top 10 Produtos mais Vendidos')
plt.xlabel('Produto')
plt.ylabel('Quantidade')
plt.xticks(rotation=80)
plt.show()

In [ ]:
# Tabela de "AMOSTRAS"
df_filtrado = filtrar_top_10_frequencias(df, 'AMOSTRA_COM_LOCAL')

# Contar as frequências dos produtos filtrados
frequencias_filtradas = df_filtrado['AMOSTRA_COM_LOCAL'].value_counts().reset_index()
frequencias_filtradas.columns = ['Produto', 'Quantidade']

# Plotar o gráfico de barras usando seaborn
plt.figure(figsize=(10, 6))
sns.barplot(x='Produto', y='Quantidade', data=frequencias_filtradas, palette='Blues_d')
plt.title('Top 10 Produtos mais Vendidos')
plt.xlabel('Produto')
plt.ylabel('Quantidade')
plt.xticks(rotation=80)
plt.show()

**4.3. Análise Multivariada**

In [ ]:
# MAPA DE CALOR
colunas_numericas = df.select_dtypes(include=['float64', 'int64'])

correlacao = colunas_numericas.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlacao, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Matriz de Correlação")
plt.show()

In [ ]:
# Filtrar os top 10 da coluna 'AMOSTRA_VENDA'
top_10_amostras = df['AMOSTRA_VENDA'].value_counts().nlargest(10).index

# Filtrar o DataFrame com as 10 principais categorias
df_top_10 = df[df['AMOSTRA_VENDA'].isin(top_10_amostras)]

# Criar o Violin Plot para os top 10
plt.figure(figsize=(10, 6))
sns.violinplot(x='AMOSTRA_VENDA', y='IDADE', data=df_top_10)

# Personalizar o gráfico
plt.title("Distribuição de Idades por Categoria de Medicamento (Top 10)")
plt.xlabel("Categoria de Medicamento")
plt.ylabel("Idade")
plt.xticks(rotation=80)  # Rotacionar os rótulos no eixo X se necessário
plt.show()

In [ ]:
# Filtrar os top 10 da coluna 'AMOSTRA_VENDA'
top_10_amostras = df['AMOSTRA_VENDA'].value_counts().nlargest(10).index

# Filtrar o DataFrame com as 10 principais categorias
df_top_10 = df[df['AMOSTRA_VENDA'].isin(top_10_amostras)]

# Criar o Violin Plot para os top 10
plt.figure(figsize=(10, 6))
sns.violinplot(x='AMOSTRA_VENDA', y='SEXO', data=df_top_10)

# Personalizar o gráfico
plt.title("Distribuição dos sexos por Categoria de Medicamento (Top 10)")
plt.xlabel("Categoria de Medicamento")
plt.ylabel("Gêneros")
plt.xticks(rotation=80)  # Rotacionar os rótulos no eixo X se necessário
plt.show()

##**5. Resultados dos Datasets**

**Dataset Anvisa-2019**

- O tratamento de Outliers não precisou ser usado, pois a maioria dos valores pareciam veridicos, porém ao analisarmos a distribuição com um gráfico Boxplot a coluna "IDADE" parecia ter alguns valores inconsistentes.

**Análise Univariada:**

- Gráfico com a frequência de vendas em cada estado(UF) do Brasil - (SP, MG, PR, GO, ES).
- Gráfico de tipos de receitas mais usadas - (1, 5, 2).
- Gráfico do Sexo do paciente para quem o medicamento era
destinado (antimicrobianos) - (3-4x mais o sexo feminino).
- Gráfico da distribuição por Idade - (30-40 anos maiores vendas).

**Análise Bivariada**

- Criando a Coluna "LOCAL" somando as tabelas "UF_VENDA" e "MUNICIPIOS_VENDA" a maior demanda foi nos municípios - (Goiânia, São Paulo, Curitiba, Belo Horizonte).
- Criando a Coluna "AMOSTRAS_VENDA" somando as tabelas "PRINCIPIO_ATIVO" e "QTD_UNIDADE_FARMACOTECNICA" - (Gabapentina, Cloridrato de Amitriptilina, C. Fluoxetina e C.Bupropiona), a (quantidade de vendas chega em torno de 5000 para cada medicamento).
- Criando a Coluna "AMOSTRA_COM_LOCAL" mostrando qual UF-Municipio onde os principios ativos foram mais vendidos - (Isotretinoína-Goiania, Cloridrato de Burpropiona-Goiania e C.Tramadol-Goiania).

**Análise Multivariada**

- Criando um Heatmap das colunas que possuem dados numéricos
- Criando um gráfico que relaciona a amostras vendidas com as idade dos pacientes - (entre 30-40 anos de idade).
- Criando um gráfico mostrando que o sexo feminino predomina, e os principios ativos de medicamentos mais comprado para essas pacientes - (Cloridrato de Amitriptilina, C. Paroxetina, C. Sertralina).

**Dataset Anvisa-2021**

- O tratamento de Outliers não precisou ser usado, pois a maioria dos valores pareciam veridicos, porém ao analisarmos a distribuição com um gráfico Boxplot a coluna "IDADE" parecia ter alguns valores inconsistentes.  

**Análise Univariada:**

- Gráfico de Barras dos estados que mais ocorreu  venda - (SP, MG, GO, PR, RJ).
- Gráfico de tipos de receitas mais usadas - (tipo 1, 5 e 2).
- O gráfico do sexo dos pacientes para quem o medicamento era destinado - (3-4x mais o sexo feminino).
- Distribuição da idade dos pacientes - (20 a 30 anos)  (aplicável apenas a medicamentos antimicrobianos).

**Análise Bivariada**

-  Criando a Coluna "LOCAL" somando as tabelas "UF_VENDA" e "MUNICIPIOS_VENDA" a maior demanda foi nos municípios - (São Paulo, Goiania e Curitiba).
- Criando a Coluna "AMOSTRAS_VENDA" somando as tabelas "PRINCIPIO_ATIVO" e "QTD_UNIDADE_FARMACOTECNICA" - (Gabapentina, Cloridrato de Amitriptilina, C. Fluoxetina e C.Bupropiona), e a (quantidade de vendas chega entre 30.000 para cada medicamento).
-Criando a Coluna "AMOSTRA_COM_LOCAL" mostrando qual UF-Municipio onde os principios ativos foram mais vendidos - (C. de Bupropiona, Isotretinoína, Topiramato - Goiânia).

**Análise Multivariada**

- Criando um Heatmap das colunas que possuem dados numéricos
- Criando um gráfico que relaciona a amostras vendidas com as idade dos pacientes - (entre 20-40 anos de idade).
- Criando um gráfico mostrando que o sexo feminino predomina, e os principios ativos de medicamentos mais comprado para essas pacientes - (Gabapentina e Cloridrato de Amitriptilina).

##**6. Analisando os Dados**

**Vendas por Estado (UF)**

- Em 2019, os estados com maior volume de vendas foram São Paulo (SP), Minas Gerais (MG), Paraná (PR), Goiás (GO) e Espírito Santo (ES). São Paulo foi, de longe, o estado com o maior número de vendas, seguido por Minas Gerais e Paraná.

- Em 2021, a distribuição das vendas por estado permaneceu similar, com São Paulo (SP) liderando novamente, seguido de Minas Gerais (MG) e Goiás (GO). No entanto, observou-se um aumento nas vendas em Goiânia (GO) e Curitiba (PR), o que pode indicar um deslocamento da demanda para essas regiões, possivelmente devido à concentração de clínicas e hospitais especializados.

**Tipos de Receitas Mais Usadas**

- Em 2019, os tipos de receita mais utilizados foram os Tipos 1, 5 e 2. O Tipo 1, que pode representar medicamentos para tratamentos crônicos, liderou as vendas, seguido dos Tipos 5 e 2, que incluem tratamentos para doenças psicológicas e antibióticos.

- Em 2021, a distribuição das receitas manteve-se similar, com o Tipo 1 novamente liderando, seguido pelos Tipos 5 e 2. Contudo, observou-se um aumento nas receitas para tratamentos psicológicos, possivelmente impulsionado pelo aumento da demanda por medicamentos relacionados à saúde mental devido aos efeitos da pandemia de COVID-19.

**Distribuição por Sexo**

- Em 2019, foi observada uma predominância do sexo feminino nas vendas de medicamentos, especialmente nos antimicrobianos, com mulheres representando de 3 a 4 vezes mais que os homens.

- Em 2021, o perfil de consumo manteve-se consistente, com o sexo feminino liderando a demanda por medicamentos, especialmente para tratamentos de doenças psicológicas, como antidepressivos (ex.: Cloridrato de Amitriptilina, Fluoxetina).

**Distribuição por Idade**

- Em 2019, a faixa etária com maior demanda por medicamentos foi a de 30 a 40 anos, especialmente para medicamentos antimicrobianos, indicando que essa faixa etária foi a mais afetada por infecções que exigem tratamentos com antibióticos.

- Em 2021, observou-se uma mudança na faixa etária predominante. A faixa etária de 20 a 30 anos foi mais relevante para medicamentos antimicrobianos, possivelmente devido a um aumento no diagnóstico e tratamento de doenças infecciosas entre jovens adultos.

**Principais Medicamentos Vendidos**

- Os medicamentos mais vendidos em 2019 foram Gabapentina, Cloridrato de Amitriptilina, Fluoxetina e Bupropiona, principalmente no tratamento de doenças neurológicas e psicológicas.

- Em 2021, os medicamentos mais vendidos mantiveram-se semelhantes, mas com destaque para a Cloridrato de Bupropiona, que teve um aumento nas vendas, associado à crescente demanda por antidepressivos e tratamentos de saúde mental.